In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

![Google Review](https://www.manyoujingyu.com/wp-content/uploads/2020/04/GOOGLE-REVIEW-banner.jpg)

# Introduction

I'm kind of people who love traveling. But sometimes I've problems like where should I visit? Are there somewhere interesting places matched with my lifestyle? Often I spent hours to search for interesting place to go out. Such a waste of time. 

What if we can build a recommender system which can recommend you several interesting venue based on your preferences. With information from Google review, I'll try to divide Google review user into cluster of similar interest for further work of building recommender system based on thier preference.

Google review is a platform where customers give a rating (from 0-5 stars) and leave a comment to a specific places in google maps

In [ ]:
cat = """user
churches
resorts
beaches
parks
theatres
museums
malls
zoo
restaurants
pubs/bars
local services
burger/pizza shops
hotels/other lodgings
juice bars
art galleries
dance clubs
swimming pools
gyms
bakeries
beauty & spas
cafes
view points
monuments
gardens"""
columns_list = cat.split("\n")
# for i in range(len(columns_list)):
#   columns_list[i] = columns_list[i].strip()

## Dataset Explanation

This data set has been sourced from the Machine Learning Repository of University of California, Irvine (UC Irvine) : Travel Review Ratings Data Set. This data set is populated by capturing user ratings from Google reviews. Reviews on attractions from 24 categories across Europe are considered. Google user rating ranges from 1 to 5 and average user rating per category is calculated.

* Attribute 1 : Unique user id 
* Attribute 2 : Average ratings on churches
* Attribute 3 : Average ratings on resorts
* Attribute 4 : Average ratings on beaches
* Attribute 5 : Average ratings on parks
* Attribute 6 : Average ratings on theatres
* Attribute 7 : Average ratings on museums
* Attribute 8 : Average ratings on malls
* Attribute 9 : Average ratings on zoo
* Attribute 10 : Average ratings on restaurants
* Attribute 11 : Average ratings on pubs/bars
* Attribute 12 : Average ratings on local services
* Attribute 13 : Average ratings on burger/pizza shops
* Attribute 14 : Average ratings on hotels/other lodgings
* Attribute 15 : Average ratings on juice bars
* Attribute 16 : Average ratings on art galleries
* Attribute 17 : Average ratings on dance clubs
* Attribute 18 : Average ratings on swimming pools
* Attribute 19 : Average ratings on gyms
* Attribute 20 : Average ratings on bakeries
* Attribute 21 : Average ratings on beauty & spas
* Attribute 22 : Average ratings on cafes
* Attribute 23 : Average ratings on view points
* Attribute 24 : Average ratings on monuments
* Attribute 25 : Average ratings on gardens


In [ ]:
df = pd.read_csv('../input/travel-review-rating-dataset/google_review_ratings.csv',header=0,names = columns_list,index_col=False)
df = df.set_index('user')
df.head()

# Exploratory Data Analysis

In [ ]:
display(df.info())
display(df.describe())

All attribute should be float64 datatype, there is something wrong with "local services" attribute. Since they're in object type. It might be error while loading file, I'll simply cast datatype to float

In [ ]:
df['local services'] = pd.to_numeric(df['local services'], errors='coerce')

In [ ]:
print('Total missing values in dataset')
display(df.isnull().sum())
df = df.dropna()

There is only 1 missing values in gardens and burger/pizza shops review. So, I'll simply drop those instance out.

Now let's look how our data is distributed

In [ ]:
sns.set(style='darkgrid',palette = 'Set3',font_scale=1.25)

In [ ]:
df_transform = pd.melt(df,value_vars=['churches', 'resorts', 'beaches', 'parks', 'theatres', 'museums',
       'malls', 'zoo', 'restaurants', 'pubs/bars', 'local services',
       'burger/pizza shops', 'hotels/other lodgings', 'juice bars',
       'art galleries', 'dance clubs', 'swimming pools', 'gyms', 'bakeries',
       'beauty & spas', 'cafes', 'view points', 'monuments', 'gardens'])

fig = plt.figure(figsize = (15,7))
g = sns.boxenplot(x='variable',y='value',data=df_transform,palette = 'Set3')
g = sns.stripplot(x='variable',y='value',data=df_transform,size=1.5, color=".6")
g.set_xticklabels(g.get_xticklabels(),rotation=90);
g.set_title(f'Overall Rating Distribution',y=1.05,fontsize=20)
g.set_xlabel("")
g.set_ylabel("Rating",labelpad = 20)
fig.tight_layout(pad = 0.5)
plt.savefig('Overall Rating Distribution.png')

**Observation**

* Overall rating are lying between 0.5 to 5 stars
* Some attractions have wide-range of rating distribution (suchas pubs/bars, restaurants) since they are common attraction point where most of tourist must visit
* On the other hand, some attrations such as gyms, bakeries, swimming pools have relatively low rating. It might be interesting to find out why most of user gave low rating to this type of attractions
* Since we don't have enough information about attraction itself or descriptive user reivews, In this notebook we'll focus on segmenting user into different cluster based on their preferences

# K-Means Clustering : Find right number of cluster

In this section, I'll try K-Means clustering algorithm on 4 Scenarios and compared it result

1. K-Means Clustering on original data (24 features)
1. K-Means Clustering on scaled origninal data (24 features with scaled) by using StandardScaler
1. K-Means Clustering on PCA component
1. K-Means Clustering on PCA component with scaled data

Let's see how it's like in action

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.decomposition import PCA
import matplotlib.ticker as ticker

#Original data
df_original = df

#Scaled Data
df_scaled = StandardScaler().fit_transform(df)

#PCA without scaling
pca = PCA(n_components = 2,random_state=42)
df_PCA = pca.fit_transform(df_original)

#PCA with scaling
df_PCA_scaled = pca.fit_transform(df_scaled)

data_list = [df_original,df_scaled,df_PCA,df_PCA_scaled]
inertia_list = []
list_k = list(range(1, 30))

#Run elbow to evaluate number of clusters
for i in range(len(data_list)):
    sse = []
    data = data_list[i]
    for k in list_k:
        km = KMeans(n_clusters=k,random_state=42)
        km.fit(data)
        sse.append(km.inertia_)
    inertia_list.append(sse)

result_ori = pd.DataFrame({'K':list_k,'Inertia':inertia_list[0],'data_type':'original'})
result_ori_scaled = pd.DataFrame({'K':list_k,'Inertia':inertia_list[1],'data_type':'original_scaled'})
result_PCA = pd.DataFrame({'K':list_k,'Inertia':inertia_list[2],'data_type':'PCA'})
result_PCA_scaled = pd.DataFrame({'K':list_k,'Inertia':inertia_list[3],'data_type':'PCA_scaled'})
result = result_ori.append(result_ori_scaled).append(result_PCA).append(result_PCA_scaled)

In [ ]:
fig = plt.figure(figsize=(12,8))
g = sns.lineplot(data=result,x='K',y='Inertia',hue='data_type',linewidth=4,marker='o')
g.set_xlabel('Nummber of cluster', labelpad = 20)
g.set_ylabel('Inertia', labelpad = 20)
g.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x/1000) + 'K'))
g.tick_params(which="both", bottom=True)
g.set_title('Selecting proper cluster number with elbow method',y=1.05, fontsize=20);

In [ ]:
pca.explained_variance_ratio_.sum()

**Observation:** 

* It's clear that PCA version with scaled data has much more lower inertia than original one and unscaled version
* Moreover, It's really hard to select appropriate number of cluster when using original data (It's have no distinguish elbow shape compared to PCA version)

However, please also note that running K-Mean clustering algorithm on only 2 principle components which represent only 34% of original information (cummilative explianed variance ratio ~ 34%) may result in not so good clustering performance


**In conclusion, I'll continue on K-Mean clustering analysis on PCA_scaled data with number of cluster = 4**


# K-Means clustering : Result and Discussion

In [ ]:
pca = PCA(n_components = 2,random_state=42)
df_PCA_scaled = pca.fit_transform(df_scaled)

model = KMeans(n_clusters=4,random_state=42)
model.fit(df_PCA_scaled)
cluster = model.labels_

In [ ]:
coeff = np.transpose(pca.components_[0:2, :])
n = coeff.shape[0]
labels = list(df.columns)


xs = df_PCA_scaled[:,0]
ys = df_PCA_scaled[:,1]
scalex = 1.0/(xs.max() - xs.min())
scaley = 1.0/(ys.max() - ys.min())

fig = plt.figure(figsize=(15,10))

#scatter plot of each data point
g = sns.scatterplot(xs * scalex,ys * scaley,hue=cluster,palette='Set2',alpha=0.8)

#add cluster centroid
plt.scatter(x=model.cluster_centers_[:,0]*scalex, y=model.cluster_centers_[:,1]*scaley, color='b',s=100,marker='X')

#add EigenVector representing how each attraction categories related to PC1 and PC2
for i in range(n):
        plt.arrow(0, 0, coeff[i,0], coeff[i,1],color = 'r',alpha = 0.5)
        plt.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, labels[i], color = 'g', ha = 'center', va = 'center')
        
        
g.set_xlabel("PC{}".format(1))
g.set_ylabel("PC{}".format(2))
g.set_title('Result of K-Means clustering',y=1.05, fontsize=20);
plt.savefig('Clustering Result.png')

K-Means clustering result in 4 clusters(segments) of user as follow,
1. Cluster#0 (Green)  : User who prefer nature like beaches and park, And also love to go to museums and theatres
1. Cluster#1 (Orange) : Fast food lover, pay attention to hotel and juice bar and also have art in thier heart
1. Cluster#2 (Light Blue) : Activities finder, this type of user love to go to local service, spend thier time in zoo, shopping in mall, having dinner in fine restaurant and pubs/bars
1. Cluster#3 (Pink) : From scatter plot, member of this cluster are not loosely spread, they might or might not share common interest 

In [ ]:
df['cluster'] = model.labels_
df_long = pd.melt(df, "cluster", var_name="categories")

fig = plt.figure(figsize = (20,10))
for i in range(len(df_long.cluster.unique())):
    plt.subplot(1,len(df_long.cluster.unique()),i+1)
    g = sns.barplot(y='categories',x='value',data=df_long[df_long['cluster']==i],palette='Set3',ci=None,estimator=np.median)
#     g.set_xticklabels(g.get_xticklabels(),rotation=90);
    g.set_title(f'Review distribution of Cluster#{i}',y=1.02,fontsize=20)
#     g.set_xlabel("")
    g.set_ylabel('Categories')
    g.set_xlim(0,5)
fig.tight_layout(pad=0.5)
plt.savefig

In [ ]:
fig = plt.figure(figsize = (20,10))
g = sns.catplot(y='categories',x='value',data=df_long,kind='bar',
                palette='Set3',ci=None,estimator=np.median,col = 'cluster',
               height = 12,aspect=0.4,
               facet_kws={'xlim':(0,5)})
fig.tight_layout(pad=0.5)
g.fig.suptitle('Review rating of each cluster',y=1.05,ha='center')
plt.savefig('Review rating of each cluster')

Above plot show median of rating given by user in different cluster. As discussed above,

* Cluster#0 : love beaches, park, theatres, museums the most
* Cluster#1 : love burger/pizza, hotel/other lodgings, juice bars  and art galleries the most
* Cluster#2 : love malls, zoo, restaurants, pub/bars, local services the most
* Cluster#3 : Although user in this group gave overall rating about 2 to all attraction, but compared to other 3 clusters, they seem to have more attention to dance clubs, swimming pools, gyms, bakeries, beauty and spas, cafes, view points, monuments and gardens than above 3 clusters. Therefore cluster#3 can be summarized as healthy and sightseeing lover.




# Conclusion

In this notebook, we've done exploratory data analysis and find out that

* Distribution of rating in each attraction categories are not the same. Some of them have wide range distribution while some are distribute narrowly in low rating region.

We also compared each data preparation methods and found out that with Stadardized data and reduced dimension with PCA. We are able to divide google review user into 4 separable groups. Even though relatively low cumulative explained varianced ratio from only 2 principle components with each cluster having following characteristics.

1. Cluster#0 (Green)  : User who prefer nature like beaches and park, And also love to go to museums and theatres
1. Cluster#1 (Orange) : Fast food lover, pay attention to hotel and juice bar and also have art in thier heart
1. Cluster#2 (Light Blue) : Activities finder, this type of user love to go to local service, spend thier time in zoo, shopping in mall, having dinner in fine restaurant and pubs/bars
1. Cluster#3 (Pink) :  Although user in this group gave overall rating about 2 to all attraction, but compared to other 3 clusters, they seem to have more attention to dance clubs, swimming pools, gyms, bakeries, beauty and spas, cafes, view points, monuments and gardens than above 3 clusters. Therefore cluster#3 can be summarized as healthy and sightseeing lover. 

With information of how different user group prefer different attraction. We can further use this information to build recommender system where we can recommend specific type of attractions to specific user to enhanced thier traveling experience and boost revenue for attraction point.


# Acknowledgement

This data set has been sourced from the Machine Learning Repository of University of California, Irvine (UC Irvine) : Travel Review Ratings Data Set 


The UCI page mentions the following publication as the original source of the data set:
Renjith, Shini, A. Sreekumar, and M. Jathavedan. 2018. Evaluation of Partitioning Clustering Algorithms for Processing Social Media Data in Tourism Domain. In 2018 IEEE Recent Advances in Intelligent Computational Systems (RAICS), 12731. IEEE